In [42]:
from scipy import misc
import numpy as np
from skimage import io
import time
import os
import socket
from urllib2 import urlopen
global f

In [73]:
def main():
    socket.setdefaulttimeout(30)
    datasetDescriptor = '../data/faces'
    textFileNames = sorted(os.listdir(datasetDescriptor))[:10]
    print textFileNames
    person = 0

    for textFileName in textFileNames:
        if textFileName.endswith('.txt'):
            person += 1
            with open(os.path.join(datasetDescriptor, textFileName), 'rt') as f:
                lines = f.readlines()[:10]
            lastLine = int(lines[-1].split(' ')[0])
            print lastLine
            dirName = textFileName.split('.txt')[0]
            classPath = os.path.join("../data/faces_images", dirName)
            if not os.path.exists(classPath):
                os.makedirs(classPath)
                lastfile = 0
            else:
                files = sorted(os.listdir(classPath))
                lastfile = int(files[-1].split('.png')[0])

            if lastLine == lastfile:
                print person, dirName, lastfile, "Done!"
                continue

            for line in lines:
                x = line.split(' ')
                fileName = x[0]
                url = x[1]
                errorLine = ''

                if lastfile < int(fileName):
                    box = np.rint(np.array(map(float, x[2:6])))
                    imagePath = os.path.join("../data/faces_images", dirName, fileName+'.png')

                    if not os.path.exists(imagePath):
                        try:
                            img = io.imread(urlopen(url,timeout = 10))
                        except Exception as e:
                            errorMessage = '{}: {}'.format(url, e)
                            errorLine = line
                        else:
                            try:
                                if img.ndim == 2:
                                    img = toRgb(img)
                                if img.ndim != 3:
                                    raise Exception('Wrong number of image dimensions')
                                hist = np.histogram(img, 255, density=True)
                                if hist[0][0] > 0.9 and hist[0][254] > 0.9:
                                    raise Exception('Image is mainly black or white')
                                else:
                                    errorMessage = 'ok!'
                                imgCropped = img[int(box[1]):int(box[3]),int(box[0]):int(box[2]),:]
                                imgResized = misc.imresize(imgCropped, (256,256))
                                misc.imsave(imagePath, imgResized)
                            except Exception as e:
                                errorMessage = '{}: {}'.format(url, e)
                                errorLine = line
                            print person,dirName,fileName,errorMessage
                with open("../data/fix/"+dirName+".txt","a") as fix:
                    if line != errorLine:
                        fix.write(line)
            print dirName + " Done!"

In [74]:
def toRgb(img):
    w, h = img.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = ret[:, :, 1] = ret[:, :, 2] = img
    return ret

if __name__ == '__main__':
    main()

['A.J._Buckley.txt', 'A.R._Rahman.txt', 'A_P_J_Abdul_Kalam.txt', 'Aamir_Khan.txt', 'Aaron_Staton.txt', 'Aaron_Tveit.txt', 'Aaron_Yoo.txt', 'Abbie_Cornish.txt', 'Abel_Ferrara.txt', 'Abigail_Breslin.txt']
10
1 A.J._Buckley 10 Done!
10
2 A.R._Rahman 10 Done!
10
3 A_P_J_Abdul_Kalam 10 Done!
10
Aamir_Khan Done!
10
Aaron_Staton Done!
10
6 Aaron_Tveit 10 Done!
10
Aaron_Yoo Done!
10
Abbie_Cornish Done!
10
Abel_Ferrara Done!
10
10 Abigail_Breslin 10 Done!
